In [162]:
from datetime import datetime, timedelta
import pandas as pd
import requests, json
from pandas import json_normalize

In [163]:
today = datetime.now()

In [164]:
today.strftime('%Y%m%y')

tyear = today.year
tmon = today.month
tday = today.day

# tmon = 4
c_date = f'{tyear:02d}-{tmon:02d}-{tday:02d}'


ser_day = datetime.strptime(c_date, '%Y-%m-%d')


In [165]:
import calendar

# 0이면 월요일, 6이면 일요일이 시작일이다.
start_day_week, end_day = calendar.monthrange(tyear, tmon)
print(start_day_week, end_day)

# 해당 달의 첫날의 datetime형식으로 변환한다.
start_date = datetime.strptime(f'{tyear:02d}-{tmon:02d}-01', '%Y-%m-%d')
end_date = datetime.strptime(f'{tyear:02d}-{tmon:02d}-{end_day:02d}', '%Y-%m-%d')
print(start_date, end_date)


0 31
2023-05-01 00:00:00 2023-05-31 00:00:00


In [166]:
start_date.strftime('%Y-%m-%d')

'2023-05-01'

In [167]:
pd.date_range(start_date.strftime('%Y-%m-%d'), periods=3)

DatetimeIndex(['2023-05-01', '2023-05-02', '2023-05-03'], dtype='datetime64[ns]', freq='D')

In [168]:
# 전 달 구하기
lines = []
l_dict = dict()

before_month_day = 0
if start_day_week < 6:
    before_month_day = (start_date - timedelta(start_day_week+1))
    print(before_month_day)

    lines.extend(pd.date_range(before_month_day.strftime('%Y-%m-%d'), periods=start_day_week+1))
    # for i in range(start_day_week+1):
    #     lines.append(before_month_day)
    #     before_month_day +=1

print(lines)
    


2023-04-30 00:00:00
[Timestamp('2023-04-30 00:00:00', freq='D')]


In [169]:
lines.extend(pd.date_range(start_date.strftime('%Y-%m-%d'), periods=end_day))

In [170]:
# lines.extend(range(1, end_day+1))
print(lines)

[Timestamp('2023-04-30 00:00:00', freq='D'), Timestamp('2023-05-01 00:00:00', freq='D'), Timestamp('2023-05-02 00:00:00', freq='D'), Timestamp('2023-05-03 00:00:00', freq='D'), Timestamp('2023-05-04 00:00:00', freq='D'), Timestamp('2023-05-05 00:00:00', freq='D'), Timestamp('2023-05-06 00:00:00', freq='D'), Timestamp('2023-05-07 00:00:00', freq='D'), Timestamp('2023-05-08 00:00:00', freq='D'), Timestamp('2023-05-09 00:00:00', freq='D'), Timestamp('2023-05-10 00:00:00', freq='D'), Timestamp('2023-05-11 00:00:00', freq='D'), Timestamp('2023-05-12 00:00:00', freq='D'), Timestamp('2023-05-13 00:00:00', freq='D'), Timestamp('2023-05-14 00:00:00', freq='D'), Timestamp('2023-05-15 00:00:00', freq='D'), Timestamp('2023-05-16 00:00:00', freq='D'), Timestamp('2023-05-17 00:00:00', freq='D'), Timestamp('2023-05-18 00:00:00', freq='D'), Timestamp('2023-05-19 00:00:00', freq='D'), Timestamp('2023-05-20 00:00:00', freq='D'), Timestamp('2023-05-21 00:00:00', freq='D'), Timestamp('2023-05-22 00:00:00'

In [171]:
# 빈 칸에 다음 달 날짜 구하기
nday = calendar.weekday(tyear, tmon, end_day)
next_cnt = 0
if nday == 6: next_cnt = 6
elif nday < 5: next_cnt = 5 - nday

# 마지막 날짜 구하기
lines.extend(pd.date_range((end_date+timedelta(1)).strftime('%Y-%m-%d'), periods=next_cnt))

# lines.extend(range(1, next_cnt+1))
# print(lines)

In [172]:
for n in lines[:5]:
    print(n)

2023-04-30 00:00:00
2023-05-01 00:00:00
2023-05-02 00:00:00
2023-05-03 00:00:00
2023-05-04 00:00:00


In [173]:
with open('static/key/holidays.txt') as f:
    apikey = f.read()

- 공휴일

In [174]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey

In [175]:
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

print(res)

{"response":{"header":{"resultCode":"00","resultMsg":"NORMAL SERVICE."},"body":{"items":{"item":[{"dateKind":"01","dateName":"1월1일","isHoliday":"Y","locdate":20230101,"seq":1},{"dateKind":"01","dateName":"설날","isHoliday":"Y","locdate":20230121,"seq":1},{"dateKind":"01","dateName":"설날","isHoliday":"Y","locdate":20230122,"seq":1},{"dateKind":"01","dateName":"설날","isHoliday":"Y","locdate":20230123,"seq":1},{"dateKind":"01","dateName":"대체공휴일","isHoliday":"Y","locdate":20230124,"seq":1},{"dateKind":"01","dateName":"삼일절","isHoliday":"Y","locdate":20230301,"seq":1},{"dateKind":"01","dateName":"어린이날","isHoliday":"Y","locdate":20230505,"seq":1},{"dateKind":"01","dateName":"부처님오신날","isHoliday":"Y","locdate":20230527,"seq":1},{"dateKind":"01","dateName":"대체공휴일","isHoliday":"Y","locdate":20230529,"seq":1},{"dateKind":"01","dateName":"현충일","isHoliday":"Y","locdate":20230606,"seq":1},{"dateKind":"01","dateName":"광복절","isHoliday":"Y","locdate":20230815,"seq":1},{"dateKind":"01","dateName":"추석","isHol

In [176]:
res = json.loads(res)
res.keys()

dict_keys(['response'])

In [177]:
res['response']['body']['items']['item']

[{'dateKind': '01',
  'dateName': '1월1일',
  'isHoliday': 'Y',
  'locdate': 20230101,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '설날',
  'isHoliday': 'Y',
  'locdate': 20230121,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '설날',
  'isHoliday': 'Y',
  'locdate': 20230122,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '설날',
  'isHoliday': 'Y',
  'locdate': 20230123,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '대체공휴일',
  'isHoliday': 'Y',
  'locdate': 20230124,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '삼일절',
  'isHoliday': 'Y',
  'locdate': 20230301,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '어린이날',
  'isHoliday': 'Y',
  'locdate': 20230505,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '부처님오신날',
  'isHoliday': 'Y',
  'locdate': 20230527,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '대체공휴일',
  'isHoliday': 'Y',
  'locdate': 20230529,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '현충일',
  'isHoliday': 'Y',
  'locdate': 20230606,
  'seq': 1},
 {'dateKind': '01',
  'd

In [178]:
holidays = json_normalize(res['response']['body']['items']['item'])
holidays

,dateKind,dateName,isHoliday,locdate,seq
0,01,1월1일,Y,20230101,1
1,01,설날,Y,20230121,1
2,01,설날,Y,20230122,1
3,01,설날,Y,20230123,1
4,01,대체공휴일,Y,20230124,1
5,01,삼일절,Y,20230301,1
6,01,어린이날,Y,20230505,1
7,01,부처님오신날,Y,20230527,1
8,01,대체공휴일,Y,20230529,1
9,01,현충일,Y,20230606,1


- 24절기

In [179]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/get24DivisionsInfo?_type=json&numOfRows=50&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

res = json.loads(res)
res.keys()

dict_keys(['response'])

In [180]:
json_normalize(res['response']['body']['items']['item'])[['dateKind', 'dateName', 'isHoliday', 'locdate']]

,dateKind,dateName,isHoliday,locdate
0,03,소한,N,20230106
1,03,대한,N,20230120
2,03,입춘,N,20230204
3,03,우수,N,20230219
4,03,경칩,N,20230306
5,03,춘분,N,20230321
6,03,청명,N,20230405
7,03,곡우,N,20230420
8,03,입하,N,20230506
9,03,소만,N,20230521


- 기념일

In [181]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getAnniversaryInfo?_type=json&numOfRows=80&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

res = json.loads(res)
dt = pd.concat([holidays, json_normalize(res['response']['body']['items']['item'])])


In [182]:
res['response']['body']['items']['item']

[{'dateKind': '02',
  'dateName': '2·28 민주운동 기념일',
  'isHoliday': 'N',
  'locdate': 20230228,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '납세자의 날',
  'isHoliday': 'N',
  'locdate': 20230303,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '3·8민주의거 기념일',
  'isHoliday': 'N',
  'locdate': 20230308,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '3·15 의거 기념일',
  'isHoliday': 'N',
  'locdate': 20230315,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '상공의 날',
  'isHoliday': 'N',
  'locdate': 20230315,
  'seq': 2},
 {'dateKind': '02',
  'dateName': '서해수호의 날',
  'isHoliday': 'N',
  'locdate': 20230324,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '4·3희생자 추념일',
  'isHoliday': 'N',
  'locdate': 20230403,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '식목일',
  'isHoliday': 'N',
  'locdate': 20230405,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '예비군의 날',
  'isHoliday': 'N',
  'locdate': 20230407,
  'seq': 1},
 {'dateKind': '02',
  'dateName': '보건의 날',
  'isHoliday': 'N',
  'locdate': 202304

In [183]:
dt[20:40]

,dateKind,dateName,isHoliday,locdate,seq
3,02,3·15 의거 기념일,N,20230315,1
4,02,상공의 날,N,20230315,2
5,02,서해수호의 날,N,20230324,1
6,02,4·3희생자 추념일,N,20230403,1
7,02,식목일,N,20230405,1
8,02,예비군의 날,N,20230407,1
9,02,보건의 날,N,20230407,2
10,02,도시농업의 날,N,20230411,1
11,02,대한민국임시정부 수립기념일,N,20230411,2
12,02,4·19 혁명 기념일,N,20230419,1


- 잡절


In [184]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getSundryDayInfo?_type=json&numOfRows=50&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

res = json.loads(res)
json_normalize(res['response']['body']['items']['item'])

,dateKind,dateName,isHoliday,locdate,seq
0,04,정월대보름,N,20230205,1
1,04,한식,N,20230406,1
2,04,단오,N,20230622,1
3,04,초복,N,20230711,2
4,04,중복,N,20230721,1
5,04,말복,N,20230810,1
6,04,칠석,N,20230822,1


- 한 번에 구하기

In [185]:
with open('static/key/holidays.txt') as f:
    apikey = f.read()
    
f_list = ['getRestDeInfo', 'getAnniversaryInfo', 'get24DivisionsInfo', 'getSundryDayInfo']
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/' #getSundryDayInfo?_type=json&numOfRows=50&solYear='
param1 = '?_type=json&numOfRows=80&solYear='
holidays = None
for name in f_list:
    url = base_url + name + param1 + str(tyear) + '&ServiceKey=' + apikey
    res = ''

    try:
        result = requests.get(url) 
        if result.status_code == 200:
            print('200')
            res = result.text
    except:
        pass

    if res: 
        res = json.loads(res)
        holidays = pd.concat([holidays, json_normalize(res['response']['body']['items']['item'])[['dateKind', 'dateName', 'isHoliday', 'locdate']] ])

200
200
200
200


In [186]:
holidays.head()

,dateKind,dateName,isHoliday,locdate
0,01,1월1일,Y,20230101
1,01,설날,Y,20230121
2,01,설날,Y,20230122
3,01,설날,Y,20230123
4,01,대체공휴일,Y,20230124


In [187]:
holidays.isna().sum().sum()

0

In [188]:
holidays['locdate'] = holidays['locdate'].astype(str)

In [189]:
# type(list(holidays['locdate'])[0])

In [190]:
# '20230101' in list(holidays['locdate'])

In [191]:
# holidays['locdate'].to_datetime()
holidays['locdate'] = pd.to_datetime(holidays['locdate'], format='%Y%m%d')

In [192]:
for dd in holidays['locdate'][:5]:
    print(dd)

2023-01-01 00:00:00
2023-01-21 00:00:00
2023-01-22 00:00:00
2023-01-23 00:00:00
2023-01-24 00:00:00


In [193]:
holidays

,dateKind,dateName,isHoliday,locdate
0,01,1월1일,Y,2023-01-01
1,01,설날,Y,2023-01-21
2,01,설날,Y,2023-01-22
3,01,설날,Y,2023-01-23
4,01,대체공휴일,Y,2023-01-24
...,...,...,...,...
2,04,단오,N,2023-06-22
3,04,초복,N,2023-07-11
4,04,중복,N,2023-07-21
5,04,말복,N,2023-08-10


In [194]:
try:
    nindex = holidays.index[holidays.dateName=='기독탄신일'][0]
    holidays.iloc[nindex, 1] = '크리스마스'
except:
    pass



In [195]:
holidays[holidays.dateName=='크리스마스']

,dateKind,dateName,isHoliday,locdate
16,01,크리스마스,Y,2023-12-25


In [196]:
holidays.locdate[:5]

0   2023-01-01
1   2023-01-21
2   2023-01-22
3   2023-01-23
4   2023-01-24
Name: locdate, dtype: datetime64[ns]

In [197]:
l_dt = pd.DataFrame(lines, columns=['locdate'])
l_dt.head()

,locdate
0,2023-04-30
1,2023-05-01
2,2023-05-02
3,2023-05-03
4,2023-05-04


In [198]:
l_dt['day'] = l_dt.locdate.apply(lambda x: int(x.strftime('%d')))


In [234]:
result_dt = pd.merge(l_dt, holidays, how='left', on='locdate')
result_dt.dateKind.fillna('05', inplace=True)
result_dt.dateName.fillna('', inplace=True)
result_dt.isHoliday.fillna('N', inplace=True)
result_dt

,day,dateKind,dateName,isHoliday
locdate,,,,
2023-04-30,30,05,,N
2023-05-01,1,02,근로자의 날,N
2023-05-02,2,05,,N
2023-05-03,3,05,,N
2023-05-04,4,05,,N
2023-05-05,5,01,어린이날,Y
2023-05-06,6,03,입하,N
2023-05-07,7,05,,N
2023-05-08,8,02,어버이 날,N


In [226]:
result_dt.groupby(['day'])['day']

In [239]:
result_dt[result_dt.duplicated()==True]

,day,dateKind,dateName,isHoliday
locdate,,,,
2023-05-30,30,05,,N
2023-06-02,2,05,,N
2023-06-03,3,05,,N


MergeError: Must pass right_on or right_index=True

In [203]:
res_list = result_dt.to_dict('records')


In [204]:
res_list


[{'locdate': Timestamp('2023-04-30 00:00:00'),
  'day': 30,
  'dateKind': '05',
  'dateName': '',
  'isHoliday': 'N'},
 {'locdate': Timestamp('2023-05-01 00:00:00'),
  'day': 1,
  'dateKind': '02',
  'dateName': '근로자의 날',
  'isHoliday': 'N'},
 {'locdate': Timestamp('2023-05-02 00:00:00'),
  'day': 2,
  'dateKind': '05',
  'dateName': '',
  'isHoliday': 'N'},
 {'locdate': Timestamp('2023-05-03 00:00:00'),
  'day': 3,
  'dateKind': '05',
  'dateName': '',
  'isHoliday': 'N'},
 {'locdate': Timestamp('2023-05-04 00:00:00'),
  'day': 4,
  'dateKind': '05',
  'dateName': '',
  'isHoliday': 'N'},
 {'locdate': Timestamp('2023-05-05 00:00:00'),
  'day': 5,
  'dateKind': '01',
  'dateName': '어린이날',
  'isHoliday': 'Y'},
 {'locdate': Timestamp('2023-05-06 00:00:00'),
  'day': 6,
  'dateKind': '03',
  'dateName': '입하',
  'isHoliday': 'N'},
 {'locdate': Timestamp('2023-05-07 00:00:00'),
  'day': 7,
  'dateKind': '05',
  'dateName': '',
  'isHoliday': 'N'},
 {'locdate': Timestamp('2023-05-08 00:00:00